# Address Parser

Is able to recognize named entities and format outputs based on a defined schema

In [1]:


GCP_PROJECT_NAME = "single-azimuth-413609"

In [2]:
!pip install --quiet --upgrade google-cloud-aiplatform 
!pip install --quiet --upgrade google-auth



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!gcloud auth application-default login --scopes=openid,https://www.googleapis.com/auth/userinfo.email,https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/sqlservice.login,https://www.googleapis.com/auth/drive

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=4oCvq3SgCR1jnwxtixGt6ZhLgBjEdr&access_type=offline&code_challenge=02T-ecmi9TvqMRQBMLwo_sVSitQqmN2SprRemd3fwfU&code_challenge_method=S256


Credentials saved to file: [/home/vincent/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "single-azimuth-413609" was added to ADC which can be used by Google client libraries for billing and quota. Note that 

In [4]:
!gcloud config set project {GCP_PROJECT_NAME}

Updated property [core/project].


In [5]:
!gcloud services enable aiplatform.googleapis.com

In [8]:
# Importing the library 
import google.auth

# Setting up credentials
credentials, _ = google.auth.default()
# authed_http = google.auth.transport.requests.AuthorizedSession(credentials)

In [9]:
import vertexai
from vertexai.preview.language_models import TextGenerationModel

vertexai.init(project=GCP_PROJECT_NAME, location="us-central1")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 126,
    "temperature": 0,
    "top_p": 1
}
model = TextGenerationModel.from_pretrained("text-bison")


## Basic french address

In [10]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 1, rue de la Paix, 75008 Paris
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "1",
  "street_type": "rue",
  "street_name": "de la Paix",
  "zip_code": "75008",
  "city": "Paris",
  "confidence": 1
}


### Unusual French address

In [11]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 78 bis Cavee des Ecameaux 76500 Elbeuf
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "78",
  "street_multiplier": "bis",
  "street_type": "Cavee",
  "street_name": "des Ecameaux",
  "zip_code": "76500",
  "city": "Elbeuf",
  "confidence": 1
}


In [12]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 78 bis Cavee des Ecameaux 76500 Elbeuf
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "78",
  "street_multiplier": "bis",
  "street_type": "Cavee",
  "street_name": "des Ecameaux",
  "zip_code": "76500",
  "city": "Elbeuf",
  "confidence": 1
}


## Complex US address

In [13]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: C/o John Doe LLC, 111, 8th Ave Ste 1509, Oklahoma, 74136-1922, USA
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "111",
  "street_multiplier": "8th",
  "street_type": "Ave",
  "street_name": "Ste 1509",
  "zip_code": "74136-1922",
  "city": "Oklahoma",
  "country": "USA",
  "confidence": 1
}


## German address 

generator at: https://tarjeta-credito.net/cvv/address-gen-germany/

In [14]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: Haßfurter Str. 1, 91056 Erlangen, Germany.
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "1",
  "street_type": "Str.",
  "street_name": "Haßfurter",
  "zip_code": "91056",
  "city": "Erlangen",
  "country": "Germany",
  "confidence": 1
}


### german address with Country not mentionned

In [15]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city and country
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: Auschnippe 9, 37170 Uslar
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "9",
  "street_name": "Auschnippe",
  "zip_code": "37170",
  "city": "Uslar",
  "country": "Germany",
  "confidence": 1
}


## Confidence level w/ False address

In [16]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: 
{
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: 
{
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: ici c'est paris !, 75001 Paris
JSON:

""",
    **parameters
)
print(f"{response.text}")

 {
  "street_number": null,
  "street_type": null,
  "street_name": null,
  "zip_code": "75001",
  "city": "Paris",
  "confidence": 0.5
}


### Confidence level w/ False address and Country not mentioned

In [17]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: 
{
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: 
{
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: 452 quater grande rue general 85000 le palais vendee
JSON:

""",
    **parameters
)
print(f"{response.text}")


 {
  "street_number": "452",
  "street_multiplier": "quater",
  "street_type": "grande rue",
  "street_name": "general",
  "zip_code": "85000",
  "city": "le palais",
  "department": "vendee",
  "country": "France",
  "confidence": 1
}


## Programmatic

Using Langchain we can have a more programmatic way 

In [110]:
!pip install --upgrade --quiet  langchain-core langchain-google-vertexai


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [112]:
from langchain_google_vertexai import VertexAI

# candidate count parameter missing ?
model = VertexAI(model_name="text-bison", top_p=1, temperature=0, max_output_tokens=126)

In [120]:

from langchain_core.pydantic_v1 import BaseModel
from typing import Optional

from langchain_core.output_parsers import JsonOutputParser


class Address(BaseModel):
    street_number: Optional[str]
    street_multiplier: Optional[str]
    street_type: Optional[str]
    street_name: Optional[str]
    locality: Optional[str]  #lieu-dit
    zip_code: Optional[str]
    city: Optional[str]
    country: Optional[str]
    confidence: int


parser = JsonOutputParser(pydantic_object=Address)
# pprint(parser.get_format_instructions())


In [118]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate

examples = [
    {
        "address": "37, rue du champ du pardon, 76000 rouen",
        "answer": """{{
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}}"""
    },
    {
        "address": "10-82 Cavée St Gervais, hameau de charette 76000 rouen",
        "answer": """{{
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"locality\": \"hameau de charette\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}}"""
    },
]

prefix = """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input"""

example_prompt = PromptTemplate(
    template="""Text: {address}\nJSON: {answer}""",
    # template="""Answer the user query.\n{format_instructions}\nText: {address}\nJSON: {answer}\n""",
    input_variables=["address", "answer"],
    # partial_variables={"format_instructions": parser.get_format_instructions()}
)

# print(example_prompt.format(**examples[0]))

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix="Text: {address}\nJSON:",
    input_variables=["address"],

)

# print(prompt.format(address="78 bis Cavee des Ecameaux 76500 Elbeuf"))


chain = prompt | model | parser

### Testing

In [121]:

address = "1 bis rue de la Paix, 75008 Paris"
Address(**chain.invoke({"address": address}))


Address(street_number='1', street_multiplier='bis', street_type='rue', street_name='de la Paix', locality=None, zip_code='75008', city='Paris', country='France', confidence=1)

In [122]:

address = "Auschnippe 9, 37170 Uslar"
Address(**chain.invoke({"address": address}))


Address(street_number='9', street_multiplier=None, street_type=None, street_name='Auschnippe', locality=None, zip_code='37170', city='Uslar', country='Germany', confidence=1)

In [24]:

address = "C/o John Doe LLC, 111, 8th Ave Ste 1509, Oklahoma, 74136-1922, USA"
add = Address(**chain.invoke({"address": address}))
add


Address(street_number='111', street_multiplier='8th', street_type='Ave', street_name='Ste 1509', zip_code='74136-1922', city='Oklahoma', country='USA', confidence=1)

### Serialize / Compress

In [25]:
import sys
from pprint import pprint
import zlib

# size of object in bytes
pprint(sys.getsizeof(add))
pprint(sys.getsizeof(add.json()))
pprint(sys.getsizeof(zlib.compress(add.json().encode())))

56
231
164


## Auto-Testing

pre-requisite: Install dependencies: pandas, pyarrow, etc...

Use of 'base nationale des Adresses' for French addresses (BAN)

#### Exploratory Data Analysis & Cleanup

In [5]:
import pandas as pd
from pprint import pprint
import numpy as np

df_addresses = pd.read_csv('./dataset/adresses-01.csv', delimiter=';', low_memory=False)


In [6]:
df_addresses.head()

,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,...,lat,type_position,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles
0,01001_4b50r5_00630,NaN,630,NaN,la Chèvre,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.132481,segment,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,LA CHEVRE,inconnue,inconnue,0,NaN
1,01001_g0ru02_00108,NaN,108,NaN,Clemencia,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.133867,parcelle,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,CLEMENCIA,cadastre,cadastre,0,01001000ZL0263
2,01001_ngzlqw_00009,NaN,9,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.152646,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN
3,01001_ngzlqw_00023,NaN,23,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.152646,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN
4,01001_ngzlqw_00026,NaN,26,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,46.152646,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN


In [9]:
df_addresses.drop(
    columns=['id_fantoir', 'code_insee', 'code_insee_ancienne_commune', 'nom_ancienne_commune', 'x', 'y', 'lon', 'lat',
             'type_position', 'source_position', 'source_nom_voie', 'certification_commune', 'cad_parcelles'],
    inplace=True)

In [104]:
df_addresses.replace(np.nan, '', regex=True, inplace=True)

In [30]:
def dataframe_row_to_pydantic_object(row: pd.Series) -> Address:
    """
    Converts a row from a Pandas DataFrame to a Pydantic object.
    Args:
        row (pd.Series): A single row from the DataFrame.
    Returns:
        SomeModel: A Pydantic object with the data from the row.
    """
    #missing street_type
    return Address(street_number=str(row['numero']), street_multiplier=row['rep'], street_name=row['nom_voie'],
                   zip_code=str(row['code_postal']), city=row['nom_commune'], confidence=1)


In [106]:
dataframe_row_to_pydantic_object(df_addresses.iloc[0])

Address(street_number='630', street_multiplier='', street_type=None, street_name='la Chèvre', zip_code='1400', city="L'Abergement-Clémenciat", country=None, confidence=1)

In [191]:
def full_address_generator(row: pd.Series) -> str:
    """
    Generates a one line address based on all the available fields from BAN
    :param row: pd.Series of one address from BAN
    :return: 
    """
    if row['nom_ld'] != '':
        row['nom_ld'] = ', ' + row['nom_ld']
    strings = [str(row['numero']), str(row['rep']), str(row['nom_voie']), str(row['nom_ld']), str(row['code_postal']),
               str(row['nom_commune'])]
    return ' '.join(filter(None, strings))
    # return f"{row['numero']}{('',' '+str(row['rep']))[row['rep'] == 'nan']}{' ' + row['nom_voie']}{('', ', '+str(row['nom_ld']))[row['nom_ld'] == 'nan']}{' '+str(row['code_postal'])}{' ' + row['nom_commune']}"

In [141]:

index = 45682
pprint(df_addresses.iloc[index])
full_address_generator(df_addresses.iloc[index])

id                              01384_tjsb7l_00004
numero                                           4
rep                                               
nom_voie                Impasse aux Vignes d'Amont
code_postal                                   1230
nom_commune                 Saint-Rambert-en-Bugey
alias                                             
nom_ld                          Hameau de Grattoux
libelle_acheminement        SAINT-RAMBERT-EN-BUGEY
nom_afnor               IMPASSE AUX VIGNES D AMONT
Name: 45682, dtype: object


"4 Impasse aux Vignes d'Amont Hameau de Grattoux 1230 Saint-Rambert-en-Bugey"

Add one line address to Dataframe

In [192]:
df_addresses['full_add'] = df_addresses.apply(lambda _row: full_address_generator(_row), axis=1)

In [193]:
df_addresses.head(50)

,id,numero,rep,nom_voie,code_postal,nom_commune,alias,nom_ld,libelle_acheminement,nom_afnor,full_add
0,01001_4b50r5_00630,630,,la Chèvre,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,LA CHEVRE,630 la Chèvre 1400 L'Abergement-Clémenciat
1,01001_g0ru02_00108,108,,Clemencia,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,CLEMENCIA,108 Clemencia 1400 L'Abergement-Clémenciat
2,01001_ngzlqw_00009,9,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,9 Imp des Epis 1400 L'Abergement-Clémenciat
3,01001_ngzlqw_00023,23,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,23 Imp des Epis 1400 L'Abergement-Clémenciat
4,01001_ngzlqw_00026,26,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,26 Imp des Epis 1400 L'Abergement-Clémenciat
5,01001_ngzlqw_00028,28,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,28 Imp des Epis 1400 L'Abergement-Clémenciat
6,01001_ngzlqw_00033,33,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,33 Imp des Epis 1400 L'Abergement-Clémenciat
7,01001_ngzlqw_00035,35,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,35 Imp des Epis 1400 L'Abergement-Clémenciat
8,01001_ngzlqw_00040,40,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,40 Imp des Epis 1400 L'Abergement-Clémenciat
9,01001_ngzlqw_00048,48,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,48 Imp des Epis 1400 L'Abergement-Clémenciat


### Testing

In [255]:
def score_ban_from_split(split: Address, ban: pd.Series) -> tuple[int, int]:
    """
    Defines a score to compare an Address object and a pd.Series of BAN
    :param split: Address object
    :param ban: pd.Series row of BAN Dataframe
    :return: score from 0 to 6 and _score one hot encoded from 0 to 111111
    """
    # print(split)
    # print(ban)
    _score = 0
    if split.street_number == str(ban['numero']): _score += 1
    if (not (bool(split.street_multiplier) | bool(ban['rep']))) | (
            split.street_multiplier == str(ban['rep'])): _score += 10
    if (split.street_type + ' ' + split.street_name) == str(ban['nom_voie']): _score += 100
    if (not (bool(split.locality) | bool(ban['nom_ld']))) | (split.locality == ban['nom_ld']): _score += 1000
    if split.zip_code == str(ban['code_postal']): _score += 10000
    if split.city == str(ban['nom_commune']): _score += 100000
    return str(_score).count('1'), _score


In [241]:
row = df_addresses.iloc[6892]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row)
score
# split_adr.locality == row['nom_ld']

street_number='140' street_multiplier=None street_type='Route' street_name='du Colombier' locality='Brénaz - Hameau de Méraléaz' zip_code='1260' city='Arvière-en-Valromey' country='France' confidence=1
id                                                       01453_0046_00140
numero                                                                140
rep                                                                      
nom_voie                                               Route du Colombier
code_postal                                                          1260
nom_commune                                           Arvière-en-Valromey
alias                                                                    
nom_ld                                        Brénaz - Hameau de Méraléaz
libelle_acheminement                                  ARVIERE-EN-VALROMEY
nom_afnor                                              ROUTE DU COLOMBIER
full_add                140 Route du Colombier , Brénaz - 

(6, 111111)

In [242]:
row = df_addresses.iloc[4564]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row)
score

street_number='50' street_multiplier=None street_type='Rue' street_name='du Prémonin' locality=None zip_code='1500' city='Ambérieu-en-Bugey' country='France' confidence=1
id                                               01004_0535_00050
numero                                                         50
rep                                                              
nom_voie                                          Rue du Prémonin
code_postal                                                  1500
nom_commune                                     Ambérieu-en-Bugey
alias                                                            
nom_ld                                                           
libelle_acheminement                            AMBERIEU-EN-BUGEY
nom_afnor                                         RUE DU PREMONIN
full_add                50 Rue du Prémonin 1500 Ambérieu-en-Bugey
Name: 4564, dtype: object


(6, 111111)

In [243]:
row = df_addresses.iloc[55213]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row)
score

street_number='2' street_multiplier=None street_type='Rue' street_name='du Chateau' locality=None zip_code='1640' city='Saint-Jean-le-Vieux' country='France' confidence=1
id                                               01363_0103_00002
numero                                                          2
rep                                                              
nom_voie                                           Rue du Chateau
code_postal                                                  1640
nom_commune                                   Saint-Jean-le-Vieux
alias                                                            
nom_ld                                                           
libelle_acheminement                          SAINT-JEAN-LE-VIEUX
nom_afnor                                          RUE DU CHATEAU
full_add                2 Rue du Chateau 1640 Saint-Jean-le-Vieux
Name: 55213, dtype: object


(6, 111111)

In [244]:
row = df_addresses.iloc[111235]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row)
score

street_number='229' street_multiplier=None street_type='Rue' street_name='des Sablonnieres' locality='Chassin' zip_code='1400' city='Neuville-les-Dames' country='France' confidence=1
id                                                       01272_0640_00229
numero                                                                229
rep                                                                      
nom_voie                                             Rue des Sablonnieres
code_postal                                                          1400
nom_commune                                            Neuville-les-Dames
alias                                                                    
nom_ld                                                            Chassin
libelle_acheminement                                   NEUVILLE-LES-DAMES
nom_afnor                                            RUE DES SABLONNIERES
full_add                229 Rue des Sablonnieres , Chassin 1400 Neuvil...
Nam

(6, 111111)

In [245]:
row = df_addresses.iloc[55466]
split_adr = Address(**chain.invoke({"address": row['full_add']}))
score = score_ban_from_split(split_adr, row)
score

street_number='147' street_multiplier=None street_type='Chemin' street_name='sous la Ville' locality=None zip_code='1640' city='Saint-Jean-le-Vieux' country='France' confidence=1
id                                                       01363_0118_00147
numero                                                                147
rep                                                                      
nom_voie                                             Chemin sous la Ville
code_postal                                                          1640
nom_commune                                           Saint-Jean-le-Vieux
alias                                                                    
nom_ld                                                                   
libelle_acheminement                                  SAINT-JEAN-LE-VIEUX
nom_afnor                                            CHEMIN SOUS LA VILLE
full_add                147 Chemin sous la Ville 1640 Saint-Jean-le-Vieux
Name: 5

(6, 111111)

### Score Compute

We need to take a portion of the data set in order to minimize the cost

In [249]:
df_addresses.shape

(265542, 11)

In [278]:
df_test = df_addresses[:100].copy()
df_test.shape

(100, 11)

In [270]:
def score_row(_row : pd.Series) -> tuple[int, int]:
    _split_adr = Address(**chain.invoke({"address": row['full_add']}))
    return score_ban_from_split(_split_adr, row)

In [265]:
score_row(df_addresses.iloc[55466])

6

In [279]:
df_test['score'] = df_test.apply(lambda _row: score_row(_row), axis=1)

In [280]:
df_test

,id,numero,rep,nom_voie,code_postal,nom_commune,alias,nom_ld,libelle_acheminement,nom_afnor,full_add,score
0,01001_4b50r5_00630,630,,la Chèvre,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,LA CHEVRE,630 la Chèvre 1400 L'Abergement-Clémenciat,"(6, 111111)"
1,01001_g0ru02_00108,108,,Clemencia,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,CLEMENCIA,108 Clemencia 1400 L'Abergement-Clémenciat,"(6, 111111)"
2,01001_ngzlqw_00009,9,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,9 Imp des Epis 1400 L'Abergement-Clémenciat,"(6, 111111)"
3,01001_ngzlqw_00023,23,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,23 Imp des Epis 1400 L'Abergement-Clémenciat,"(6, 111111)"
4,01001_ngzlqw_00026,26,,Imp des Epis,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,26 Imp des Epis 1400 L'Abergement-Clémenciat,"(6, 111111)"
...,...,...,...,...,...,...,...,...,...,...,...,...
95,01001_0175_00272,272,,Route des Garaudes,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,ROUTE DES GARAUDES,272 Route des Garaudes 1400 L'Abergement-Cléme...,"(6, 111111)"
96,01001_0175_00288,288,,Route des Garaudes,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,ROUTE DES GARAUDES,288 Route des Garaudes 1400 L'Abergement-Cléme...,"(6, 111111)"
97,01001_0175_00295,295,,Route des Garaudes,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,ROUTE DES GARAUDES,295 Route des Garaudes 1400 L'Abergement-Cléme...,"(6, 111111)"
98,01001_0175_00327,327,,Route des Garaudes,1400,L'Abergement-Clémenciat,,,L'ABERGEMENT-CLEMENCIAT,ROUTE DES GARAUDES,327 Route des Garaudes 1400 L'Abergement-Cléme...,"(6, 111111)"
